In [1]:
#from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

import streamlit as st
openai_api_key = st.secrets["OPENAI_API_KEY"]

# Select Model

#content_model_name = 'gpt-3.5-turbo'
#content_model_name = 'gpt-3.5-turbo-16k'
content_model_name = 'gpt-4'

#code_model_name = 'gpt-3.5-turbo'
#code_model_name = 'gpt-3.5-turbo-16k'
code_model_name = 'gpt-4'

# Set model temperature
content_model_temperature = 0.0
code_model_temperature = 0.0

class OpenAI_Chat_Model:
    def __init__(self, model_name, temperature, api_key):
        self.model_name = model_name
        self.temperature = temperature
        self.api_key = api_key
        self.prompt = None
        self.resp onse = None
        

    def get_response(self, prompt):
        self.model = OpenAI(model_name = self.model_name, temperature = self.temperature, openai_api_key = self.api_key)
        self.prompt = prompt
        self.response = self.model(prompt)
        return self.response

    def get_report(self):
        report = {
            'model_name': self.model_name,
            'temperature': self.temperature,
            'prompt': self.prompt,
            'response': self.response
        }
        return report

In [2]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate

from backend.models import UI_INSTRUCTIONS_FORMAT, CONTENT_INSTRUCTIONS_FORMAT

content_parser = PydanticOutputParser(pydantic_object=CONTENT_INSTRUCTIONS_FORMAT)
code_parser = PydanticOutputParser(pydantic_object=UI_INSTRUCTIONS_FORMAT)

In [3]:
content_query = "I want to learn about dinosaurs, but I don't know where to start."

In [4]:
content_prompt_template = PromptTemplate(
    template = '''
        You are a a highly knowegeable assitant having a conversation with a curious person.
        Your job is to educate them about the topic they are interested in. 

        {format_instructions}\n
        {query}\n''',
        input_variables=["query"],
        partial_variables={"format_instructions": content_parser.get_format_instructions()},
    )

content_prompt = content_prompt_template.format_prompt(query=content_query).to_string()

In [5]:
content_model = OpenAI_Chat_Model(model_name = content_model_name, temperature = content_model_temperature, api_key = openai_api_key)

In [6]:
content_response = content_model.get_response(prompt=content_prompt)

/Users/jamisonproctor/Documents/dev/intelligent_ui/venv/lib/python3.11/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/jamisonproctor/Documents/dev/intelligent_ui/venv/lib/python3.11/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [7]:
content_response_dict = dict(content_parser.parse(content_response))

In [8]:
content_response_dict

{'Overview': "Well, buckle up, because we're about to embark on a journey 230 million years back in time, to the Mesozoic Era! This was the age of the dinosaurs, a period that lasted for over 160 million years. Dinosaurs were a diverse group of reptiles, and they came in all shapes and sizes, from the mighty T-Rex, king of the predators, to the gentle, long-necked Brachiosaurus. 🦖🦕\n\nDinosaurs are classified into two main groups based on their hip structure: the Saurischia, which includes creatures like the T-Rex and the long-tailed, long-necked sauropodomorphs, and the Ornithischia, herbivores with bird-like hips. 🍖🌿\n\nBut it wasn't just land that these magnificent creatures dominated; there were also marine reptiles like the Plesiosaur, and flying reptiles like the Pterosaur. 🌊🌤️\n\nSadly, all good things must come to an end. The reign of the dinosaurs ended with a bang - literally - when a massive asteroid hit the earth about 65 million years ago. This event, known as the K-T exti

In [9]:
code_query = content_response_dict['Overview']

In [10]:
code_prompt = PromptTemplate(
    template = '''
        You are a super creative teacher. You are very creative and have a lot of ideas.
        You are great at helping people explore topics in a natural way.
        You always use a lot of emojis to make your content more engaging.
        
        You will help to create a UI which will help the reader to further explore the topic.

        Only respond using the format_instructions.\n
        
        {format_instructions}\n
        {query}\n''',
        input_variables=["query"],
        partial_variables={"format_instructions": code_parser.get_format_instructions()},
    )

formated_code_prompt = code_prompt.format_prompt(query=code_query).to_string()

In [11]:
code_model = OpenAI_Chat_Model(model_name = code_model_name, temperature = code_model_temperature, api_key = openai_api_key)

In [12]:
code_response = code_model.get_response(prompt=formated_code_prompt)
code_response_dict = dict(code_parser.parse(code_response))

code_response_dict

{'ui_element_1_type': 'SelectSlider',
 'ui_element_1_label': 'Choose your favorite dinosaur era 🦖',
 'ui_element_1_options': ['Triassic', 'Jurassic', 'Cretaceous'],
 'ui_element_1_min_value': None,
 'ui_element_1_max_value': None,
 'ui_element_2_type': 'Checkbox',
 'ui_element_2_label': 'Select the types of dinosaurs you want to learn about 🦕',
 'ui_element_2_options': ['Saurischia 🍖',
  'Ornithischia 🌿',
  'Marine Reptiles 🌊',
  'Flying Reptiles 🌤️'],
 'ui_element_2_min_value': None,
 'ui_element_2_max_value': None,
 'ui_element_3_type': 'TextInput',
 'ui_element_3_label': "What's your question about dinosaurs? 🤔",
 'ui_element_3_options': None,
 'ui_element_3_min_value': None,
 'ui_element_3_max_value': None}

In [13]:
code_response


'{\n  "ui_element_1_type": "SelectSlider",\n  "ui_element_1_label": "Choose your favorite dinosaur era 🦖",\n  "ui_element_1_options": ["Triassic", "Jurassic", "Cretaceous"],\n  "ui_element_1_min_value": null,\n  "ui_element_1_max_value": null,\n  "ui_element_2_type": "Checkbox",\n  "ui_element_2_label": "Select the types of dinosaurs you want to learn about 🦕",\n  "ui_element_2_options": ["Saurischia 🍖", "Ornithischia 🌿", "Marine Reptiles 🌊", "Flying Reptiles 🌤️"],\n  "ui_element_2_min_value": null,\n  "ui_element_2_max_value": null,\n  "ui_element_3_type": "TextInput",\n  "ui_element_3_label": "What\'s your question about dinosaurs? 🤔",\n  "ui_element_3_options": null,\n  "ui_element_3_min_value": null,\n  "ui_element_3_max_value": null\n}'